In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Feb 23 10:59:46 2020

@author: Christo Strydom
"""
import tensorflow as tf
from tensorflow import keras

import os
import tempfile
import datetime as datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
os.chdir('/media/lnr-ai/christo/github_repos/ae_mp/')
# from sklearn.model_selection import train_test_split
# import pandas as pd
# import numpy as np
#%%
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
%matplotlib inline

In [52]:
ae_data_path='/media/lnr-ai/christo/github_repos/ae_mp/data/'
ae_candles_filename = 'data/USDZAR_Candlestick_5_M_BID_01.01.2019-08.02.2020_wrangled.csv'
ae_df=pd.read_csv(ae_candles_filename)
ae_df=ae_df[ae_df.is_business_day].copy()

In [53]:
def pkl_dump(df, fname, data_path):
   f = open("{data_path}{fname}.pkl".format(fname=fname,data_path=data_path),"wb")
   pickle.dump(df,f)
   f.close()
   return

def pkl_load(fname, data_path):
   return pickle.load( open( "{data_path}{fname}.pkl".format(fname=fname, data_path=data_path), "rb" ) )  

def time_filter_fn(df, start_int_seconds, end_int_seconds):
   return (ae_df['int_seconds']>=start_int_seconds) & (ae_df['int_seconds']<=end_int_seconds)

def businessday_filter_fn(df):
   return df['is_business_day']

def mp_filter_fn(df, start_int_seconds, end_int_seconds):
   time_filter=time_filter_fn(df, start_int_seconds, end_int_seconds)
   businessday_filter=businessday_filter_fn(df)
   return time_filter&businessday_filter

def days_fn(df,mp_filter):
   return list(set(df.loc[mp_filter, 'date'])) 
# days=list(set(ae_df.loc[mp_filter, 'date'])) 

def day_filter_fn(df,day):
   return df['date']==day

def range_fn(df, day_filter,time_filter):
    f=np.logical_and(day_filter, time_filter)
    on_high=max(df.loc[f, 'high'])
    on_low=min(df.loc[f, 'low'])   
    on_range=on_high-on_low
    return on_high, on_low, on_range

def on_fn(df, day_filter, on_high, on_low, on_range):
   df.loc[day_filter, 'on_low']=on_low
   df.loc[day_filter, 'on_high']=on_high
   df.loc[day_filter, 'on_range']=on_range
   return df

def order_fn(df,day_filter, perc_level, on_high, on_low, on_range):   
   df.loc[day_filter, 'sell_{perc_level}'.format(perc_level=perc_level)]=on_high+perc_level*on_range/100
   df.loc[day_filter, 'buy_{perc_level}'.format(perc_level=perc_level)]=on_low-perc_level*on_range/100
   return df

def trade_range_fn(df, day):
    day_filter=day_filter_fn(df, day)
    time_filter=time_filter_fn(df, 0, 28800)
    on_high, on_low, on_range=range_fn(df, day_filter, time_filter)
    df=on_fn(df, day_filter, on_high, on_low, on_range)
    for perc_level in [50,75,100]:
        df=order_fn(df=df,day_filter=day_filter, 
                  perc_level=perc_level, 
                  on_high=on_high, 
                  on_low=on_low, 
                  on_range=on_range)
    return df

In [54]:
#%%
# ae_df['jhb_timestamp']==2019-01-01 02:05:00
list(ae_df)
ae_df_list=['datetime',
 'date',
 'Gmt time',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'bidopen',
 'bidclose',
 'bidhigh',
 'bidlow',
 'askopen',
 'askclose',
 'askhigh',
 'asklow',
 'tickqty',
 'open',
 'high',
 'low',
 'close',
 'period',
 'dom',
 'awdn',
 'month',
 'doy',
 'wny',
 'timestamp',
 'ny_timestamp',
 'london_timestamp',
 'jhb_timestamp',
 'int_seconds',
 'universaldate',
 'is_business_day',
 'previous_business_day',
 'five_previous_business_day']
format_str=MP_FXCM_DATETIME_FORMAT='%Y-%m-%d %H:%M:%S'
ae_df['jhb_timestamp'] =  pd.to_datetime(ae_df['jhb_timestamp'], format=format_str)
format_str='%Y-%m-%d'
ae_df['date'] =  pd.to_datetime(ae_df['date'], format=format_str)
# ae_df['overnight_range']=False
# time_filter=(ae_df['int_seconds']>=0) & (ae_df['int_seconds']<=28800)
# businessday_filter=ae_df['is_business_day']
# mp_filter=time_filter&businessday_filter

In [55]:
mp_filter=mp_filter_fn(df=ae_df, start_int_seconds=0, end_int_seconds=28800)
ae_df['overnight_range']=mp_filter
days = days_fn(ae_df,mp_filter)
for day in days:
   index=days.index(day)
   print('Doing day = {day} and index = {index}...'.format(index=index, day=day))
   ae_df=trade_range_fn(ae_df, day)

Doing day = 2019-11-20 00:00:00 and index = 0...
Doing day = 2020-01-20 00:00:00 and index = 1...
Doing day = 2019-11-05 00:00:00 and index = 2...
Doing day = 2019-03-18 00:00:00 and index = 3...
Doing day = 2019-06-12 00:00:00 and index = 4...
Doing day = 2019-12-10 00:00:00 and index = 5...
Doing day = 2019-03-13 00:00:00 and index = 6...
Doing day = 2019-02-06 00:00:00 and index = 7...
Doing day = 2019-05-15 00:00:00 and index = 8...
Doing day = 2019-09-06 00:00:00 and index = 9...
Doing day = 2019-06-20 00:00:00 and index = 10...
Doing day = 2019-10-18 00:00:00 and index = 11...
Doing day = 2019-11-29 00:00:00 and index = 12...
Doing day = 2019-03-14 00:00:00 and index = 13...
Doing day = 2019-06-03 00:00:00 and index = 14...
Doing day = 2019-09-23 00:00:00 and index = 15...
Doing day = 2019-06-14 00:00:00 and index = 16...
Doing day = 2019-08-15 00:00:00 and index = 17...
Doing day = 2019-07-04 00:00:00 and index = 18...
Doing day = 2019-10-21 00:00:00 and index = 19...
Doing day 

Doing day = 2019-01-11 00:00:00 and index = 163...
Doing day = 2019-02-04 00:00:00 and index = 164...
Doing day = 2019-05-03 00:00:00 and index = 165...
Doing day = 2019-01-14 00:00:00 and index = 166...
Doing day = 2019-05-29 00:00:00 and index = 167...
Doing day = 2020-01-06 00:00:00 and index = 168...
Doing day = 2020-01-09 00:00:00 and index = 169...
Doing day = 2020-01-22 00:00:00 and index = 170...
Doing day = 2019-01-16 00:00:00 and index = 171...
Doing day = 2019-10-07 00:00:00 and index = 172...
Doing day = 2020-01-24 00:00:00 and index = 173...
Doing day = 2020-01-01 00:00:00 and index = 174...
Doing day = 2019-05-13 00:00:00 and index = 175...
Doing day = 2019-06-04 00:00:00 and index = 176...
Doing day = 2019-08-21 00:00:00 and index = 177...
Doing day = 2019-02-14 00:00:00 and index = 178...
Doing day = 2019-02-28 00:00:00 and index = 179...
Doing day = 2019-10-28 00:00:00 and index = 180...
Doing day = 2019-12-26 00:00:00 and index = 181...
Doing day = 2019-10-15 00:00:00

In [56]:
pkl_dump(df=ae_df, fname='ae_df.csv', data_path=ae_data_path)

In [57]:
ae_df=pkl_load(fname='ae_df.csv', data_path=ae_data_path)

In [58]:
day_filter=day_filter_fn(ae_df, day)

In [59]:
ae_df[ae_df.is_business_day]

,datetime,date,Gmt time,Open,High,Low,Close,Volume,bidopen,bidclose,...,overnight_range,on_low,on_high,on_range,sell_50,buy_50,sell_75,buy_75,sell_100,buy_100
0,2019-01-01 00:00:00,2019-01-01,01.01.2019 00:00:00.000,14.33822,14.33822,14.33822,14.33822,0.0,14.33822,14.33822,...,True,14.33822,14.33822,0.00000,14.338220,14.338220,14.338220,14.338220,14.33822,14.33822
1,2019-01-01 00:05:00,2019-01-01,01.01.2019 00:05:00.000,14.33822,14.33822,14.33822,14.33822,0.0,14.33822,14.33822,...,True,14.33822,14.33822,0.00000,14.338220,14.338220,14.338220,14.338220,14.33822,14.33822
2,2019-01-01 00:10:00,2019-01-01,01.01.2019 00:10:00.000,14.33822,14.33822,14.33822,14.33822,0.0,14.33822,14.33822,...,True,14.33822,14.33822,0.00000,14.338220,14.338220,14.338220,14.338220,14.33822,14.33822
3,2019-01-01 00:15:00,2019-01-01,01.01.2019 00:15:00.000,14.33822,14.33822,14.33822,14.33822,0.0,14.33822,14.33822,...,True,14.33822,14.33822,0.00000,14.338220,14.338220,14.338220,14.338220,14.33822,14.33822
4,2019-01-01 00:20:00,2019-01-01,01.01.2019 00:20:00.000,14.33822,14.33822,14.33822,14.33822,0.0,14.33822,14.33822,...,True,14.33822,14.33822,0.00000,14.338220,14.338220,14.338220,14.338220,14.33822,14.33822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112668,2020-02-07 23:35:00,2020-02-07,07.02.2020 23:35:00.000,15.05669,15.05669,15.05669,15.05669,0.0,15.05669,15.05669,...,False,14.87436,14.99079,0.11643,15.049005,14.816145,15.078113,14.787037,15.10722,14.75793
112669,2020-02-07 23:40:00,2020-02-07,07.02.2020 23:40:00.000,15.05669,15.05669,15.05669,15.05669,0.0,15.05669,15.05669,...,False,14.87436,14.99079,0.11643,15.049005,14.816145,15.078113,14.787037,15.10722,14.75793
112670,2020-02-07 23:45:00,2020-02-07,07.02.2020 23:45:00.000,15.05669,15.05669,15.05669,15.05669,0.0,15.05669,15.05669,...,False,14.87436,14.99079,0.11643,15.049005,14.816145,15.078113,14.787037,15.10722,14.75793
112671,2020-02-07 23:50:00,2020-02-07,07.02.2020 23:50:00.000,15.05669,15.05669,15.05669,15.05669,0.0,15.05669,15.05669,...,False,14.87436,14.99079,0.11643,15.049005,14.816145,15.078113,14.787037,15.10722,14.75793


In [60]:
day

Timestamp('2019-01-10 00:00:00')

In [61]:
np.where(day_filter)

(array([2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026,
        2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037,
        2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048,
        2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059,
        2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070,
        2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081,
        2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092,
        2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100, 2101, 2102, 2103,
        2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112, 2113, 2114,
        2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 2124, 2125,
        2126, 2127, 2128, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 2136,
        2137, 2138, 2139, 2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147,
        2148, 2149, 2150, 2151, 2152, 2153, 2154, 2155, 2156, 2157, 2158,
        2159, 2160, 2161, 2162, 2163, 

In [62]:
df=ae_df[day_filter].copy()

In [63]:
day

Timestamp('2019-01-10 00:00:00')

In [64]:
time_filter

0          True
1          True
2          True
3          True
4          True
          ...  
112956    False
112957    False
112958    False
112959    False
112960    False
Name: int_seconds, Length: 112961, dtype: bool

In [65]:
time_filter=time_filter_fn(ae_df, 0, 28800)

In [66]:
len(np.where(day_filter)[0])

288

In [67]:
on_high, on_low, on_range=range_fn(df=ae_df, day_filter=day_filter, time_filter=time_filter)

In [68]:
df=on_fn(df, day_filter, on_high, on_low, on_range)

In [69]:
for perc_level in [50,75,100]:
    df=order_fn(df=df,day_filter=day_filter, 
              perc_level=perc_level, 
              on_high=on_high, 
              on_low=on_low, 
              on_range=on_range)

In [70]:
list(ae_df)

['datetime',
 'date',
 'Gmt time',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'bidopen',
 'bidclose',
 'bidhigh',
 'bidlow',
 'askopen',
 'askclose',
 'askhigh',
 'asklow',
 'tickqty',
 'open',
 'high',
 'low',
 'close',
 'period',
 'dom',
 'awdn',
 'month',
 'doy',
 'wny',
 'timestamp',
 'ny_timestamp',
 'london_timestamp',
 'jhb_timestamp',
 'int_seconds',
 'universaldate',
 'is_business_day',
 'previous_business_day',
 'five_previous_business_day',
 'overnight_range',
 'on_low',
 'on_high',
 'on_range',
 'sell_50',
 'buy_50',
 'sell_75',
 'buy_75',
 'sell_100',
 'buy_100']

In [71]:
df

,datetime,date,Gmt time,Open,High,Low,Close,Volume,bidopen,bidclose,...,overnight_range,on_low,on_high,on_range,sell_50,buy_50,sell_75,buy_75,sell_100,buy_100
2592,2019-01-10 00:00:00,2019-01-10,10.01.2019 00:00:00.000,13.85707,13.85707,13.84017,13.84317,34.00,13.85707,13.84317,...,True,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2593,2019-01-10 00:05:00,2019-01-10,10.01.2019 00:05:00.000,13.84327,13.84806,13.84187,13.84767,49.50,13.84327,13.84767,...,True,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2594,2019-01-10 00:10:00,2019-01-10,10.01.2019 00:10:00.000,13.84777,13.84856,13.84627,13.84667,23.00,13.84777,13.84667,...,True,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2595,2019-01-10 00:15:00,2019-01-10,10.01.2019 00:15:00.000,13.84767,13.85377,13.84627,13.85337,168.50,13.84767,13.85337,...,True,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2596,2019-01-10 00:20:00,2019-01-10,10.01.2019 00:20:00.000,13.85297,13.85497,13.84777,13.85407,303.25,13.85297,13.85407,...,True,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2019-01-10 23:35:00,2019-01-10,10.01.2019 23:35:00.000,13.86313,13.86313,13.85247,13.86312,688.80,13.86313,13.86312,...,False,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2876,2019-01-10 23:40:00,2019-01-10,10.01.2019 23:40:00.000,13.86313,13.86313,13.85157,13.86213,883.50,13.86313,13.86213,...,False,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2877,2019-01-10 23:45:00,2019-01-10,10.01.2019 23:45:00.000,13.86213,13.86413,13.85156,13.86212,868.60,13.86213,13.86212,...,False,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2878,2019-01-10 23:50:00,2019-01-10,10.01.2019 23:50:00.000,13.86213,13.86313,13.85157,13.86313,627.10,13.86213,13.86313,...,False,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992


In [72]:
trade_range_fn(df, day)

,datetime,date,Gmt time,Open,High,Low,Close,Volume,bidopen,bidclose,...,overnight_range,on_low,on_high,on_range,sell_50,buy_50,sell_75,buy_75,sell_100,buy_100
2592,2019-01-10 00:00:00,2019-01-10,10.01.2019 00:00:00.000,13.85707,13.85707,13.84017,13.84317,34.00,13.85707,13.84317,...,True,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2593,2019-01-10 00:05:00,2019-01-10,10.01.2019 00:05:00.000,13.84327,13.84806,13.84187,13.84767,49.50,13.84327,13.84767,...,True,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2594,2019-01-10 00:10:00,2019-01-10,10.01.2019 00:10:00.000,13.84777,13.84856,13.84627,13.84667,23.00,13.84777,13.84667,...,True,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2595,2019-01-10 00:15:00,2019-01-10,10.01.2019 00:15:00.000,13.84767,13.85377,13.84627,13.85337,168.50,13.84767,13.85337,...,True,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2596,2019-01-10 00:20:00,2019-01-10,10.01.2019 00:20:00.000,13.85297,13.85497,13.84777,13.85407,303.25,13.85297,13.85407,...,True,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2019-01-10 23:35:00,2019-01-10,10.01.2019 23:35:00.000,13.86313,13.86313,13.85247,13.86312,688.80,13.86313,13.86312,...,False,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2876,2019-01-10 23:40:00,2019-01-10,10.01.2019 23:40:00.000,13.86313,13.86313,13.85157,13.86213,883.50,13.86313,13.86213,...,False,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2877,2019-01-10 23:45:00,2019-01-10,10.01.2019 23:45:00.000,13.86213,13.86413,13.85156,13.86212,868.60,13.86213,13.86212,...,False,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992
2878,2019-01-10 23:50:00,2019-01-10,10.01.2019 23:50:00.000,13.86213,13.86313,13.85157,13.86313,627.10,13.86213,13.86313,...,False,13.8175,13.90508,0.08758,13.94887,13.77371,13.970765,13.751815,13.99266,13.72992


In [73]:
list(ae_df.on_low)[0]


14.338220000000002

In [74]:
dst=list(df['datetime'])[-1]
# dst.year
date_object = datetime.datetime.strptime(dst, "%Y-%m-%d %H:%M:%S")

In [75]:
date_object.year

2019

In [76]:
d=datetime.datetime(date_object.year, date_object.month,date_object.day, 8, 30)

In [77]:
d.strftime("%Y-%m-%d %H:%M:%S")

'2019-01-10 08:30:00'

In [78]:
def time_fn(dst):
    date_object = datetime.strptime(dst, "%Y-%m-%d %H:%M:%S")
    end_d=datetime(date_object.year, date_object.month,date_object.day, 8,0)
    start_d=datetime(date_object.year, date_object.month,date_object.day, 0, 0)
    fin_d=datetime(date_object.year, date_object.month,date_object.day, 23,59,59)
    return start_d.strftime("%Y-%m-%d %H:%M:%S"), end_d.strftime("%Y-%m-%d %H:%M:%S"), fin_d.strftime("%Y-%m-%d %H:%M:%S")

In [79]:
list(df)

['datetime',
 'date',
 'Gmt time',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'bidopen',
 'bidclose',
 'bidhigh',
 'bidlow',
 'askopen',
 'askclose',
 'askhigh',
 'asklow',
 'tickqty',
 'open',
 'high',
 'low',
 'close',
 'period',
 'dom',
 'awdn',
 'month',
 'doy',
 'wny',
 'timestamp',
 'ny_timestamp',
 'london_timestamp',
 'jhb_timestamp',
 'int_seconds',
 'universaldate',
 'is_business_day',
 'previous_business_day',
 'five_previous_business_day',
 'overnight_range',
 'on_low',
 'on_high',
 'on_range',
 'sell_50',
 'buy_50',
 'sell_75',
 'buy_75',
 'sell_100',
 'buy_100']

In [88]:
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
df=ae_df[day_filter].copy()
min_x=min(df['Low'])
max_x=max(df['High'])
fig = go.Figure(data=[go.Candlestick(x=df['datetime'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
# start_time=list(df['datetime'])[0]
# end_time=list(df['datetime'])[-1]
start_time, end_time, fin_time = time_fn(dst=list(df['datetime'])[0])
fig.update_layout(
    title='Overnight trade',
    yaxis_title='USDZAR',
    shapes = [
        dict(
        x0=end_time, x1=fin_time, y0=list(df.sell_100)[0], y1=list(df.sell_100)[0],
        line=dict(color="red",width=1,dash="dashdot"),opacity=0.2),
        dict(
        x0=end_time, x1=fin_time, y0=list(df.sell_75)[0], y1=list(df.sell_75)[0],
        line=dict(color="red",width=1,dash="dashdot"),opacity=0.3),
        dict(
        x0=end_time, x1=fin_time, y0=list(df.sell_50)[0], y1=list(df.sell_50)[0],
        line=dict(color="red",width=1,dash="dashdot"),opacity=0.4),
        dict(
        x0=end_time, x1=fin_time, y0=list(df.buy_100)[0], y1=list(df.buy_100)[0],
        line=dict(color="green",width=1,dash="dashdot"),opacity=0.2),
        dict(
        x0=end_time, x1=fin_time, y0=list(df.buy_75)[0], y1=list(df.buy_75)[0],
        line=dict(color="green",width=1,dash="dashdot"),opacity=0.3),
        dict(
        x0=end_time, x1=fin_time, y0=list(df.buy_50)[0], y1=list(df.buy_50)[0],
        line=dict(color="green",width=1,dash="dashdot"),opacity=0.4),      
        dict(
        x0=end_time, x1=end_time, y0=list(df.on_low)[0], y1=list(df.on_high)[0],
        line_width=1),
        dict(
        x0=start_time, x1=end_time, y0=list(df.on_low)[0], y1=list(df.on_low)[0],
        line_width=1),
        dict(
        x0=start_time, x1=end_time, y0=list(df.on_high)[0], y1=list(df.on_high)[0],
        line_width=1)])

fig.add_shape(
        # filled Rectangle
            type="rect",
            xref="x",
            yref="y",    
            x0=start_time,
            y0=df.buy_100,
            x1=end_time,
            y1=df.sell_100,
            line=dict(
                color="RoyalBlue",
                width=2,
            ),
            fillcolor="LightSkyBlue",
        )
fig.add_shape(
        # Rectangle reference to the plot
            type="rect",
            xref="paper",
            yref="paper",
            x0=0.25,
            y0=0,
            x1=0.5,
            y1=0.5,
            line=dict(
                color="LightSeaGreen",
                width=3,
            ),
            fillcolor="PaleTurquoise",
        )

# fig.add_shape(
#         # Line reference to the axes
#             type="line",
#             xref="x",
#             yref="y",
#             x0=4,
#             y0=0,
#             x1=8,
#             y1=1,
#             line=dict(
#                 color="LightSeaGreen",
#                 width=3,
#             ),
#         )
# fig.update_yaxes(range=[min_x, max_x])
#     shapes = [dict(
#         x0='2019-08-21 00:00:00', x1='2019-08-21 23:55:00', y0=list(df.on_high)[0], y1=list(df.on_high)[0],
#         line_width=1)],
# fig.update_layout(
#     title='The Great Recession',
#     yaxis_title='USDZAR',
# #     shapes = [dict(
# #         x0='2019-08-21 00:00:00', x1='2019-08-21 23:55:00', y0=list(df.on_low)[0], y1=list(df.on_low)[0],
# #         line_width=1)],
    
#     shapes = [dict(
#         x0='2019-08-21 00:00:00', x1='2019-08-21 23:55:00', y0=list(df.on_high)[0], y1=list(df.on_high)[0],
#         line_width=1)])
    
#     annotations=[dict(
#         x='2016-12-09', y=0.05, xref='x', yref='paper',
#         showarrow=False, xanchor='left', text='Increase Period Begins')]


fig.show()